In [1]:
%pip install pandas pyodbc

   ---------------------------------------- 0.0/68.9 kB ? eta -:--:--
   ----------------- ---------------------- 30.7/68.9 kB ? eta -:--:--
   ---------------------------------------- 68.9/68.9 kB 751.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os

# Define the folder containing the CSV files
folder_path = "db_csv"

# Iterate over all files in the folder
for file_name in os.listdir(folder_path):
    # Check if the file name contains the pattern to remove
    if file_name.endswith("_202103042023.csv"):
        # Construct the new file name
        new_name = file_name.replace("_202103042023", "")
        
        # Build full paths for renaming
        old_file_path = os.path.join(folder_path, file_name)
        new_file_path = os.path.join(folder_path, new_name)
        
        # Rename the file
        os.rename(old_file_path, new_file_path)
        print(f"Renamed: {file_name} -> {new_name}")

print("Renaming complete!")


In [117]:
import pandas as pd
import pyodbc

def csv_to_ssms_table(csv_file_path, server, database, encoding='windows-1255'):
    """
    Reads a CSV file into a pandas DataFrame and inserts its contents into a SQL Server table.
    
    :param csv_file_path: Path to the CSV file. The file name (without extension) should match the table name.
    :param server: SQL Server name or address.
    :param database: Database name.
    :param encoding: Encoding of the CSV file (default is 'utf-8').
    """
    # Extract table name from the CSV file name
    table_name = csv_file_path.split('/')[-1].split('.')[0]
    
    # Read the CSV file into a pandas DataFrame
    try:
        df = pd.read_csv(csv_file_path, encoding=encoding)
    except UnicodeDecodeError:
        print(f"Failed to decode {csv_file_path} with encoding '{encoding}'. Please try a different encoding.")
        return

    # Drop rows with any null values
    df.dropna(inplace=True)
    conn_str = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};trusted_connection=yes;"
    
    # Connect to the SQL Server
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    # Generate SQL INSERT query template
    columns = ', '.join(df.columns)
    placeholders = ', '.join(['?'] * len(df.columns))
    insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    
    try:
        # Insert each row into the table
        for index, row in df.iterrows():
            cursor.execute(insert_query, tuple(row))
        
        # Commit the transaction
        conn.commit()
        print(f"Data from {csv_file_path} successfully inserted into {table_name}")
    except Exception as e:
        print(f"Error inserting data into {table_name}: {e}")
        conn.rollback()
    finally:
        # Close the connection
        cursor.close()
        conn.close()


In [118]:
csv_file_path = "db_csv/KNS_Law.csv"
sqlserver = "DESKTOP-9LFS960\SQLEXPRESS"
database = "BetaKnesset"
csv_to_ssms_table(csv_file_path, sqlserver, database)


Data from db_csv/KNS_Law.csv successfully inserted into KNS_Law


In [99]:
import chardet

with open('db_csv/KNS_Agenda.csv', 'rb') as f:
    result = chardet.detect(f.read())
    print(result)


{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}
